In [36]:
import pyes
import elasticsearch

es_address='140.118.155.14:9200'
conn = pyes.es.ES(es_address)
bq = pyes.query.BoolQuery() 
#q = pyes.query.MatchAllQuery()

cve = pyes.query.TermQuery(field="text", value="cve")
vul = pyes.query.TermQuery(field="text", value="vulnerability")
exp = pyes.query.TermQuery(field="text", value="exploit")
back = pyes.query.TermQuery(field="text", value="backdoor")
apt = pyes.query.TermQuery(field="text", value="apt")
mal = pyes.query.TermQuery(field="text", value="malware")


ESR = pyes.ESRange(field="created_at_linux_timestamp", from_value="1388534400000", to_value="1420070400000", 
                   include_lower=True ,include_upper=False)
rq = pyes.query.RangeQuery(qrange=ESR)

bq.add_should(cve)
bq.add_should(vul)
bq.add_should(exp)
bq.add_should(back)
bq.add_should(apt)
bq.add_should(mal)
bq.add_must(rq)

result = conn.search(query=bq , indices='twitter2' , doc_types='tweet') 
len(result)

3822

In [37]:
from collections import defaultdict
twitterPerMon = defaultdict(set)
tweetsPerMon = [0]*12
indicators = defaultdict(list)

In [38]:
import datetime
import re

haveUrl = 0
regex = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
for oneTweet in result:
    month = int(datetime.datetime.fromtimestamp(oneTweet['created_at_linux_timestamp']/1000).strftime('%Y-%m-%d %H:%M:%S').split('-')[1])
    twitterPerMon[month - 1].add(oneTweet['user']['id'])
    tweetsPerMon[month - 1] += 1
    if regex.findall(oneTweet['text']):
        #print(regex.findall(oneTweet['text']))
        haveUrl += 1
    if oneTweet['user']['id'] not in indicators.iterkeys():
        indicators[oneTweet['user']['id']].append(1)
        indicators[oneTweet['user']['id']].append(0)
        indicators[oneTweet['user']['id']].append(0)
    else:
        indicators[oneTweet['user']['id']][0] += 1
        indicators[oneTweet['user']['id']][1] += oneTweet['retweet_count']
        indicators[oneTweet['user']['id']][2] += oneTweet['favorite_count']

noUrl = len(result) - haveUrl
#print (twitterPerMon)
#print (tweetsPerMon)
#print (haveUrl, noUrl)
print (indicators)
len(indicators)

defaultdict(<type 'list'>, {17775619: [5, 53, 63], 18789893: [214, 2907, 787], 297856522: [153, 3571, 1359], 21501463: [1, 0, 0], 8236572: [28, 1391, 40], 40072739: [23, 51, 6], 41260072: [169, 4104, 157], 15943215: [29, 427, 24], 18783289: [24, 1194, 7], 14244412: [1, 0, 0], 19206209: [161, 1087, 317], 14780493: [97, 1945, 1444], 1102: [6, 11, 13], 4905551: [3, 2, 0], 23791544: [1, 0, 0], 14174808: [3, 12, 0], 20492381: [24, 960, 4], 312383587: [14, 972, 0], 15637093: [10, 102, 32], 17604714: [16, 105, 14], 21650546: [15, 1091, 4], 813697: [17, 885, 4], 302716548: [39, 52, 22], 17226373: [26, 360, 6], 57279114: [21, 169, 2], 198365324: [47, 470, 137], 118059149: [365, 2763, 1275], 810641: [45, 2307, 0], 40713876: [1, 0, 0], 161038997: [112, 229, 45], 8917142: [7, 318, 5], 11791512: [148, 2510, 414], 14090906: [107, 2632, 530], 3247471: [20, 193, 10], 14669471: [55, 3977, 698], 86064583: [44, 49, 3], 97450156: [20, 134, 82], 14415986: [8, 26, 4], 14666934: [31, 2875, 971], 390189754: [

91

In [7]:
from bokeh.charts import BoxPlot, Donut, Bar, output_notebook, show

# best support is with data in a format that is table-like
data = {
    'tweets': tweetsPerMon,
    'label' : ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"],
    'twitter' : [len(twitter) for index, twitter in twitterPerMon.items() ]
}
print (data)

twitter = Bar(data, values='twitter', label='label', title="Twitter Per Month")

tweet = Bar(data, values='tweets', label='label', agg='mean', title="Tweets Per Month")


output_notebook()
show(twitter)
show(tweet)

{'tweets': [322, 316, 226, 285, 194, 271, 352, 260, 404, 424, 399, 369], 'twitter': [45, 48, 40, 55, 47, 49, 58, 54, 64, 68, 59, 60], 'label': ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']}


Loading BokehJS ...

In [8]:
data = {
    'label' : ['haveUrl ' + str( float(haveUrl) / len(result) * 100) + "%", 'noUrl ' + str( float(noUrl) / len(result) * 100) + "%"],
    'value' : [haveUrl, noUrl]
}


pie = Donut(data, label='label', values='value', text_font_size='13pt', width=500)


output_notebook()

show(pie)

Loading BokehJS ...

In [9]:
bq2 = pyes.query.BoolQuery() 
bq2.add_should(cve)
bq2.add_must(rq)
justcve = conn.search(query=bq2 , indices='twitter2' , doc_types='tweet')
len(justcve)

265

In [10]:
twitterMonthCVE = defaultdict(list)
for oneTweet in justcve:
    month = int(datetime.datetime.fromtimestamp(oneTweet['created_at_linux_timestamp']/1000).strftime('%Y-%m-%d %H:%M:%S').split('-')[1])
    twitterMonthCVE[month - 1].append(oneTweet['user']['id'])
print (twitterMonthCVE)

defaultdict(<type 'list'>, {0: [43130563, 18783289, 15655289, 15655289, 18783289], 1: [17775619, 43130563, 39262054, 39262054, 390189754, 15589731, 43130563, 43130563, 43130563, 19206209, 39262054, 135907568, 14090906, 135907568, 15655289, 8236572, 19206209], 2: [15589731, 15757132, 43130563, 135907568, 15589731, 15589731, 15655289, 19206209, 118059149, 118059149, 118059149, 15637093, 118059149, 118059149, 15581551, 118059149, 39262054, 118059149, 43130563], 3: [43130563, 15589731, 43130563, 18252252, 18789893, 14090906, 18783289, 19206209, 11687162, 19206209, 118059149, 11687162, 118059149, 17767238, 14090906, 118059149, 14415986, 65845659, 43130563, 118059149, 41260072], 4: [15943215, 118059149, 17775619, 15589731, 312383587, 390189754, 118059149, 15655289, 43130563, 23566038, 19206209, 118059149, 43130563, 312383587, 19206209, 390189754, 43130563], 5: [43130563, 15589731, 15943215, 15589731, 15589731, 15589731, 15581551, 41260072, 118059149, 19206209, 118059149, 13089682, 135907568,

In [11]:
from collections import Counter
twitterMonthCountCVE = defaultdict(list)
for key, values in twitterMonthCVE.items():
    twitterMonthCountCVE['month'] += [key + 1] * len(dict(Counter(values)).values())
    twitterMonthCountCVE['account'] += dict(Counter(values)).values()
print (twitterMonthCountCVE)

defaultdict(<type 'list'>, {'account': [2, 1, 2, 2, 1, 3, 2, 1, 4, 1, 1, 1, 1, 1, 3, 1, 1, 1, 7, 1, 1, 2, 1, 2, 1, 1, 1, 1, 4, 1, 3, 1, 2, 1, 1, 2, 1, 2, 1, 2, 3, 1, 3, 1, 1, 2, 1, 4, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 3, 2, 1, 1, 2, 3, 1, 1, 1, 1, 6, 1, 1, 1, 1, 5, 3, 2, 2, 1, 1, 3, 1, 1, 4, 2, 2, 5, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 10, 1, 2, 1, 9, 3, 1, 1, 2, 1, 1, 4, 1, 1, 1, 1, 6, 1, 1, 1, 1, 5, 1, 4, 1, 8, 1, 2, 3, 1, 1, 1, 4, 2, 1, 1, 2], 'month': [1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12]})


In [12]:
box = BoxPlot(twitterMonthCountCVE, values='account', label='month', title="Twitter CVE BoxPlot")

output_notebook()

show(box)

Loading BokehJS ...